This notebook analyzes the data for version 6 of the paper.

In [1]:
%matplotlib inline

In [2]:
from kid_readout.measurement.io import nc
from kid_readout.analysis.timeseries import binning
from kid_readout.measurement import acquire
print(acquire.git_log())

commit e4e34522012e3f5d2b657f0213c2490c3d68915f
Author: Daniel Flanigan <daniel.isaiah.flanigan@gmail.com>
Date:   Mon Aug 29 14:15:49 2016 -0400

    small enhancements: auto-cast to MeasurementList, error messages, and docstrings



In [3]:
data_directory = '/data/readout/'

shield_files = ['2016-07-13_104209_magnetic_shield.nc',
                '2016-07-26_185334_magnetic_shield.nc']

ambient_files = ['2016-07-13_144753_ambient.nc',
                 '2016-07-13_160639_ambient.nc']

array_files = ['2016-07-14_125121_magnet_array.nc',  # 162 mm, up
               '2016-07-14_191847_magnet_array.nc',  # 65 mm, up
               '2016-07-15_001146_magnet_array.nc',  # 100 mm, up
               '2016-07-15_123235_magnet_array.nc',  # 120 mm, up
               '2016-07-15_165543_magnet_array.nc',  # 140 mm, up
               '2016-07-16_140542_magnet_array.nc',  # 150 mm, up
               '2016-07-18_135051_magnet_array.nc',  # 85 mm, up
               '2016-07-18_184047_magnet_array.nc',  # 130 mm, up; saved as 85 mm and retroactively fixed
               '2016-07-19_143115_magnet_array.nc',  # 145 mm, up
               '2016-07-19_200237_magnet_array.nc',  # 155 mm, up
               '2016-07-20_141430_magnet_array.nc',  # 155 mm, up
               '2016-07-20_155530_magnet_array.nc',  # cool 155 mm, up; data 85 mm, up
               '2016-07-20_191253_magnet_array.nc',  # 85 mm, up
               '2016-07-20_204043_magnet_array.nc',  # cool 85 mm, up; data 155 mm, up
               '2016-07-22_130630_magnet_array.nc',  # 65 mm, down
               '2016-07-22_173850_magnet_array.nc',  # 0 mm, up
               '2016-07-26_111004_magnet_array.nc',  # 0 mm, down
              ]

In [4]:
# This frequency was empirically determined
f_ptc = 1.412

# These values are pasted from the notebook 2016-08-26_magnet_array_field_vs_distance
up_offset = 2.5346229684359303
up_slope = 466848717.86876786
down_offset = -3.4036232503921622
down_slope = -465226998.86899847

In [ ]:
def magnet_field_uT(d_mm, orientation):
    if orientation == 1:
        return up_slope / d_mm**3 + up_offset
    elif orientation == -1:
        return down_slope / d_mm**3 + down_offset
    else:
        raise ValueError("Invalid orientation: {}".format(orientation))


def harmonic_mask(f, fundamental, num_harmonics, width):
    mask = np.ones(f.size, dtype=np.bool)
    for harmonic in range(1, num_harmonics + 1):
        indices = np.abs(f - fundamental * harmonic) < width
        mask[indices] = False
    return mask


def log_bin_noise_after_masking_harmonics(frequency, data, fundamental=f_ptc, num_harmonics=15, width=0.05):
    mask = harmonic_mask(f=frequency, fundamental=fundamental, num_harmonics=num_harmonics, width=width)
    frequency_masked = frequency[mask]
    data_masked = data[mask]
    log_bin_edges = binning.log_bin_edges(frequency_masked, bins_per_decade=30, ensure_none_empty=False)
    bin_indices = np.digitize(frequency_masked, log_bin_edges)
    frequency_binned = np.array([frequency_masked[bin_indices==n].mean() for n in range(1, bin_indices.size)])
    data_binned = np.array([data_masked[bin_indices==n].mean() for n in range(1, bin_indices.size)])
    nan_mask = np.logical_not(np.isnan(frequency_binned))
    return frequency_binned[nan_mask], data_binned[nan_mask]


def sss_to_series(sss, shield, ambient, altered):
    series = sss.to_dataframe().iloc[0]

    # Calculate the magnetic field
    series['shield'] = shield
    series['ambient'] = ambient
    series['altered'] = altered
    if shield:
        series['ambient_field_uT'] = 0
    else:
        series['ambient_field_uT'] = -30
    if shield or ambient:
        series['field_uT'] = series.ambient_field_uT
    else:
        if 'magnet_array_orientation' not in series:
            series['magnet_array_orientation'] = 1
        series['optical_plate_to_stage_top_mm'] = 335
        series['stage_top_to_chip_mm'] = 4.82
        series['jack_top_to_magnet_center_mm'] = 25.5 + 1.5
        if not altered:
            series['magnet_distance_mm'] = (series.optical_plate_to_stage_top_mm + series.stage_top_to_chip_mm -
                                            (series.magnet_array_height_mm + series.jack_top_to_magnet_center_mm))
            series['magnet_field_uT'] = magnet_field_uT(series.magnet_distance_mm, series.magnet_array_orientation)
            series['field_uT'] = series.ambient_field_uT + series.magnet_field_uT
    
    # Calculate the noise spectra after masking pulse tube harmonics
    sss.set_S(detrend=plt.mlab.detrend_none, window=plt.mlab.window_hanning, binned=False, NFFT=sss.stream.s21_raw.size)
    S_frequency_masked_binned, S_xx_masked_binned = log_bin_noise_after_masking_harmonics(sss.S_frequency, sss.S_xx)
    series['S_xx_masked_binned'] = S_xx_masked_binned
    S_frequency_masked_binned, S_yy_masked_binned = log_bin_noise_after_masking_harmonics(sss.S_frequency, sss.S_yy)
    series['S_yy_masked_binned'] = S_yy_masked_binned
    series['S_frequency_masked_binned'] = S_frequency_masked_binned
    return series

# Select, analyze, and save data

In [ ]:
attenuation = 41
numbers = [0, 1, 2]

file_names = [shield_files[1], ambient_files[1]] + array_files
node_names = len(file_names) * ['SweepStreamArray13']

for file_name, node_name in zip(file_names, node_names):
    print(file_name)
    ncf = nc.NCFile(data_directory + file_name)
    ssa = ncf.read(node_name)
    ncf.close()
    assert ssa.stream_array.roach_state.dac_attenuation == attenuation
    shield = 'magnetic_shield_bucket' in ssa.state
    ambient = not shield and 'magnet_array' not in ssa.state
    altered = not shield and not ambient and 'cooling_height_mm' in ssa.state.magnet_array
    serieses = [sss_to_series(ssa[number], shield=shield, ambient=ambient, altered=altered) for number in numbers]
    df = pd.concat(serieses, axis=1, ignore_index=True).T
    npy_file_name = '{}_num_{}_atten_{}.npy'.format(file_name, '_'.join([str(number) for number in numbers]), attenuation)
    np.save(npy_file_name, df.to_records())

2016-07-26_185334_magnetic_shield.nc


/home/flanigan/miniconda2/envs/analysis/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
